In [1]:
# Importing necessary libraries
import json
import requests
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Dropout, LSTM
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.metrics import mean_absolute_error, accuracy_score

In [2]:
# Fetching data from the API
response = requests.get('https://min-api.cryptocompare.com/data/histoday?fsym=ETH&tsym=CAD&limit=1000')
json.loads(response.content)['Data']

In [3]:
# Creating DataFrame from fetched data
df = pd.DataFrame(json.loads(response.content)['Data'])
df.head()

In [4]:
# Setting 'time' as index
df = df.set_index('time')
df.head()

In [5]:
# Retaining only necessary columns
df = df[['high', 'low', 'open', 'volumefrom', 'volumeto', 'close']]
df.head()

In [6]:
# Function to split data into training and testing sets
def train_test_split(df):
    split_row = len(df) - int(0.1 * len(df))
    train_data = df.iloc[:split_row]
    test_data = df.iloc[split_row:]
    return train_data, test_data
train, test = train_test_split(df)

In [7]:
# Function to extract windowed data
def extract_windows_data(df):
    window_data = []
    for idx in range(len(df) - 10):
        tmp = df[idx: (idx + 10)].copy()
        window_data.append(tmp.values)
    return np.array(window_data)

In [8]:
# Function to prepare data
def prepare_data(df):
    train_data, test_data = train_test_split(df)
    X_train = extract_windows_data(train_data)
    X_test = extract_windows_data(test_data)
    y_train = train_data['close'][10:].values
    y_test = test_data['close'][10:].values
    y_train = train_data['close'][:-10].values
    y_test = test_data['close'][:-10].values
    return train_data, test_data, X_train, X_test, y_train, y_test

In [9]:
# Preparing data
df.iloc[:-1].reset_index(drop=True)

In [10]:
# Preparing data continued
df.iloc[1:].reset_index(drop=True)

In [11]:
# Preparing data continued
df.iloc[1:].reset_index(drop=True) / df.iloc[:-1].reset_index(drop=True)

In [12]:
# Preparing data continued
df2 = df.iloc[1:].reset_index(drop=True) / df.iloc[:-1].reset_index(drop=True)
train, test, X_train, X_test, y_train, y_test = prepare_data(df2)

In [13]:
# Displaying sample target value from training data
y_train[0]

In [14]:
# Displaying sample input data from training data
X_train[0]

In [15]:
# Converting target values to binary classification labels
y_train[y_train > 1] = 1
y_train[y_train < 1] = 0
y_train

In [16]:
# Converting target values to binary classification labels for testing data
y_test[y_test > 1] = 1
y_test[y_test < 1] = 0
y_test

In [17]:
# Building LSTM model
model = Sequential()
model.add(LSTM(100, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.5))
model.add(Dense(units=1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [18]:
# Fitting the model
history = model.fit(X_train, y_train, epochs=200, batch_size=32, shuffle=True, verbose=1)

In [19]:
# Making predictions
preds = model.predict(X_test).squeeze()

In [20]:
# Converting predictions to binary classification labels
preds[preds > 0.5] = 1
preds[preds <= 0.5] = 0
preds

In [21]:
# Displaying true values from testing data
y_test

In [22]:
# Calculating accuracy
accuracy_score(y_test, preds)